In [ ]:
# based on https://observablehq.com/@fil/contour-labels-canvas
# and      https://observablehq.com/@fil/contour-labels-svg

In [ ]:
import json
import numpy as np


In [ ]:
with open('./data.json') as f:
    data = json.load(f)

In [ ]:
values = np.array(data['values'], dtype='float32')
values = values.reshape((data['height'], data['width']))
values

In [ ]:
values.shape

In [ ]:
import numpy as np
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot_image_gl import ImageGL, Contour
import ipywidgets as widgets
scale_x = LinearScale(min=0, max=1)
scale_y = LinearScale(min=0, max=1)
scales = {'x': scale_x, 'y': scale_y}
axis_x = Axis(scale=scale_x, label='x')
axis_y = Axis(scale=scale_y, label='y', orientation='vertical')
scales_image = {'x': scale_x, 'y': scale_y, 'image': ColorScale(min=np.min(values).item(), max=np.max(values).item())}


In [ ]:
figure = Figure(scales=scales, axes=[axis_x, axis_y])
image = ImageGL(image=values, scales=scales_image)
contour = Contour(image=image, level=180, scales=scales_image)
figure.marks = (image, contour)

In [ ]:
figure

In [ ]:
slider = widgets.FloatSlider(value=contour.level, min=np.min(values).item(), max=np.max(values).item())
widgets.jslink((slider, 'value'), (contour, 'level'))
slider

In [ ]:
cp = widgets.ColorPicker(value='purple')
widgets.jslink((cp, 'value'), (contour, 'color'))
cp